In [ ]:
import pandas as pd

df = pd.read_csv("food_rows.csv")
df.head()

In [ ]:
df.describe()
df.isnull().sum()
df.duplicated().sum()

In [ ]:
## change formatting for better embeddings
data = df.to_dict("records")

In [ ]:
%pip install qdrant-client
%pip install sentence-transformers

In [ ]:
import os

Gemini_api_key = os.getenv("GEMINI_API_KEY")

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
qdrant = QdrantClient(":memory:")

In [ ]:
qdrant.create_collection(
    collection_name="food_rows",
    vectors_config=models.VectorParams(size=encoder.get_sentence_embedding_dimension(), distance=models.Distance.COSINE)

)

In [ ]:
qdrant.upload_points(
   collection_name="food_rows",
   points=[
      models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["name"]).tolist(),
            payload=doc
      )
      for idx, doc in enumerate(data)
   ]
)

In [ ]:

user_prompt=input("Enter a prompt: ")

In [ ]:
hits = qdrant.search(
   collection_name="food_rows",
   query_vector=encoder.encode(user_prompt).tolist(),
   limit=5  # Return 5 closest points
)


In [ ]:
search_results = [hit.payload for hit in hits]

In [ ]:
%pip install -q -U google-genai

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=Gemini_api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=f"You are a helpful assistant. And you will use this context to answer : {search_results}."),
    contents=user_prompt
)

print(response.text)
